In [1]:
import os

In [2]:
%pwd

'd:\\project\\Predict-Lung-Disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\Predict-Lung-Disease'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [16]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [17]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.pt",
            training_data="artifacts/data_split/test",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        
        return eval_config

In [18]:
import torch
from pathlib import Path
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models  

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_loader(self):
        transform = transforms.Compose([
            transforms.Resize(self.config.params_image_size[:-1]),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))  # Normalize (mean, std)
        ])

        dataset = datasets.ImageFolder(root=self.config.training_data, transform=transform)
        self.valid_loader = DataLoader(dataset, batch_size=self.config.params_batch_size, shuffle=False)

    @staticmethod
    def load_model(path: Path) -> torch.nn.Module:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = models.vgg16()
        model.load_state_dict(torch.load(path, map_location=device))  
        model.to(device)
        model.eval()
        return model

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_loader()

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        criterion = torch.nn.CrossEntropyLoss()  # Adjust based on your model
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in self.valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                total_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        self.score = {"loss": total_loss / len(self.valid_loader), "accuracy": correct / total}

    def save_score(self):
        save_json(path=Path("scores.json"), data=self.score)

In [21]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e

[2024-11-18 10:23:00,109: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-18 10:23:00,114: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-18 10:23:00,116: INFO: common: creating directory at: artifacts]


C:\Users\Account_Test\AppData\Local\Temp\ipykernel_7200\3504987693.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(path, map_location=d

KeyboardInterrupt: 